In [1]:
import pandas as pd
import json
import random

In [2]:
df = pd.read_csv('../../data/mimic-acute-mi.csv')
df[df['STEMI']==1].head()

,hadm_id,ecg_time,st_elevation,st_depression,t_wave,Acute_MI,study_id,text,valuenum,comments,troponin,STEMI,NSTEMI
27,25600836.0,2103-09-05 00:07:00,1,0,0,1,46339089,Allergies: \nPatient recorded as having No Kno...,0.06,CTROPNT > 0.10 NG/ML SUGGESTS ACUTE MI.,1,1,0
104,29197570.0,2110-01-31 04:40:00,1,0,0,1,43919386,Allergies: \nPatient recorded as having No Kno...,0.39,CTROPNT > 0.10 NG/ML SUGGESTS ACUTE MI.,1,1,0
107,29197570.0,2110-01-31 15:42:00,1,0,0,1,41200609,Allergies: \nPatient recorded as having No Kno...,0.62,CTROPNT > 0.10 NG/ML SUGGESTS ACUTE MI.,1,1,0
279,20480421.0,2110-02-28 22:46:00,1,1,0,1,44431440,Allergies: \nNo Known Allergies / Adverse Drug...,3.22,cTropnT > 0.10 ng/mL suggests Acute MI.,1,1,0
355,20139992.0,2110-03-19 10:34:00,1,0,0,1,44767134,Allergies: \nPatient recorded as having No Kno...,0.46,CTROPNT > 0.10 NG/ML SUGGESTS ACUTE MI.,1,1,0


In [3]:
with open('../../data/conversations_new.json') as f:
    conversations = json.load(f)

In [4]:
# Function to filter and print the conversation by specific ID
def print_conversation_by_id(conversations, target_id):
    for item in conversations:
        if item.get('id') == target_id:
            print(f"Conversation with ID {target_id}:")
            print(json.dumps(item, indent=4))
            print("\n" + "-"*40 + "\n")
            return  # Exit after finding the conversation
    print(f"No conversation found with ID {target_id}")

# Example usage:
target_id = 43919386  # Replace this with the desired ID
print_conversation_by_id(conversations, target_id)

Conversation with ID 43919386:
{
    "id": 43919386,
    "image": "43919386.jpeg",
    "conversations": [
        {
            "from": "human",
            "value": "<image>\nAllergies: \nPatient recorded as having No Known Allergies to Drugs\n \nChief Complaint:\nChest Pain\n \nMajor Surgical or Invasive Procedure:\nCardiac Catheterization. no intervention\n\n \nHistory of Present Illness:\nPatient is ___ y.o. man w/ pmh of Hypertension, hyperlipidemia, \nPast Medical History:\nHTN  \nHyperlipidemia  \nR knee meniscal repair  \nmelanoma removed from right thigh ___ \n\n \nSocial History:\n___\nFamily History:\nFather died in his ___ of an MI.  \n \nMedications on Admission:\nZocor 20 mg daily  \nCozaar 50 mg daily\n\n \n\n Kindly analyze the ECG. Determine whether the patient has ST elevation."
        },
        {
            "from": "gpt",
            "value": "The ECG displays evidence of ST elevation."
        },
        {
            "from": "human",
            "value": " Kindl

In [5]:
N = 5  # Number of elements to display

for i, item in enumerate(conversations[-5:], start=1):
    print(f"Element {i}:")
    print(json.dumps(item, indent=4))
    print("\n" + "-"*40 + "\n")

Element 1:
{
    "id": 41095220,
    "image": "41095220.jpeg",
    "conversations": [
        {
            "from": "human",
            "value": "<image>\nAllergies: \nAmiodarone / Vasotec / Iodine-Iodine Containing / Cardizem / \nALDDOMET / clonidine / IVP DYE  / donepezil\n \nChief Complaint:\nBilateral foot pain\n \nMajor Surgical or Invasive Procedure:\nNone\n\n \nHistory of Present Illness:\nMs. ___ is a ___ woman\nwith CAD, HFrEF EF ___ s/p ICD placement, a. fib not on\nanticoagulation and Alzheimer's presenting with worsening\nbilateral foot pain and ulceration. \n\nPast Medical History:\n- Hypertension\n- Diabetes, on insulin  \n- Paroxysmal atrial fibrillation, not on anticoagulation  \n- Polymorphic VT following treatment with amiodarone\n- Subsequent dual-chamber ___ ICD implant with most  \nrecent generator change in ___. Her device is a ___ \nVirtuoso with a Sprint Fidelis lead, which is on advisory.  \n- EF ___ with 3+ MR, 4+ TR on echo from ___.  \n- Dilated cardiomyopa

In [6]:
with open('../../data/conv_templates.json') as f:
    conv_templates = json.load(f)

In [7]:
df = df[~df['text'].isna()]
df = df[(df['troponin'] & ~df['valuenum'].isnull() & ~df['comments'].isnull()) | (df['troponin'] == 0)]

In [8]:
conversations = []

for idx, row in df.iterrows():
    idx_1 = row['study_id']
    conv = {
        "id": row['study_id'],
        "image": f"{idx_1}.jpeg",
        "conversations": []
    }

    if row['STEMI']:
        conv['diagnosis'] = 'STEMI'
    elif row['NSTEMI']:
        conv['diagnosis'] = 'NSTEMI'
    else:
        conv['diagnosis'] = 'NO ACS'

    conv['conversations'].append({
        "from": "human",
        "value" : random.choice(conv_templates['st_elev_q']).replace('{INSERT PATIENT NOTE}', row['text'])
    })

    if row['st_elevation']:
        conv['conversations'].append({
            "from": "gpt",
            "value" : random.choice(conv_templates['st_elev_a_true'])
        })
        conv['conversations'].append({
            "from": "human",
            "value" : random.choice(conv_templates['st_depres_twi_q']).replace('Your prior assessment was: {INSERT PRIOR ANSWER}','') #conv['conversations'][-1]['value']
        })
        if row['st_depression'] or row['t_wave']:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['st_depres_twi_a_true'])
            })
            conv['conversations'].append({
                "from": "human",
                "value" : random.choice(conv_templates['troponin_q']).replace('Your prior assessment was: {INSERT PRIOR ANSWER}', '') #conv['conversations'][-1]['value']
            })
        else:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['st_depres_twi_a_false'])
            })
            conv['conversations'].append({
                "from": "human",
                "value" : random.choice(conv_templates['troponin_q']).replace('Your prior assessment was: {INSERT PRIOR ANSWER}','') #conv['conversations'][-1]['value']
            })
        
        if row['troponin']:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['troponin_a_true'])
            })
            conv['conversations'].append({
                "from": "human",
                "value" : random.choice(conv_templates['diag_q_trop']).replace('Your prior assessment was: {INSERT PRIOR ANSWER}', '').replace('{INSERT_RESULT}', str(row['valuenum']) + ',' + row['comments'])
            })
        else:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['troponin_a_false'])
            })
            conv['conversations'].append({
                "from": "human",
                "value" : random.choice(conv_templates['diag_q']).replace('Your prior assessment was: {INSERT PRIOR ANSWER}','') #conv['conversations'][-1]['value']
            })

        if row['STEMI']:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['diag_a_stemi'])
            })
        else:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['diag_a_normal'])
            })
    else:
        conv['conversations'].append({
            "from": "gpt",
            "value" : random.choice(conv_templates['st_elev_a_false'])
        })
        conv['conversations'].append({
            "from": "human",
            "value" : random.choice(conv_templates['st_depres_twi_q']).replace('Your prior assessment was: {INSERT PRIOR ANSWER}','') #conv['conversations'][-1]['value']
        })
        if row['st_depression'] or row['t_wave']:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['st_depres_twi_a_true'])
            })
            conv['conversations'].append({
                "from": "human",
                "value" : random.choice(conv_templates['troponin_q']).replace('Your prior assessment was: {INSERT PRIOR ANSWER}','') #conv['conversations'][-1]['value']
            })
        else:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['st_depres_twi_a_false'])
            })
            conv['conversations'].append({
                "from": "human",
                "value" : random.choice(conv_templates['troponin_q']).replace('Your prior assessment was: {INSERT PRIOR ANSWER}','') #conv['conversations'][-1]['value']
            })

        if row['troponin']:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['troponin_a_true'])
            })
            conv['conversations'].append({
                "from": "human",
                "value" : random.choice(conv_templates['diag_q_trop']).replace('Your prior assessment was: {INSERT PRIOR ANSWER}','').replace('{INSERT_RESULT}', str(row['valuenum']) + ',' + row['comments'])  #conv['conversations'][-1]['value']
            })
        else:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['troponin_a_false'])
            })
            conv['conversations'].append({
                "from": "human",
                "value" : random.choice(conv_templates['diag_q']).replace('Your prior assessment was: {INSERT PRIOR ANSWER}','') #conv['conversations'][-1]['value']
            })
            
        if row['NSTEMI']:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['diag_a_nstemi'])
            })
        else:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['diag_a_normal'])
            })
    conversations.append(conv)


In [9]:
with open('../../data/conversations_new.json', 'w') as f:
    json.dump(conversations, f)
